# Becoming backprop ninja

In [93]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from fontTools.unicodedata import block
%matplotlib inline

In [94]:
words = open('../names.txt','r').read().split()
words[:3]

['emma', 'olivia', 'ava']

In [95]:
# building the vocab of words and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(vocab_size)
itos

27


{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [96]:
block_size = 3
def build_dataset(words):
    X,y = [],[]
    for w in words:
        context = [0]*block_size
        for ch in w +'.':
            ix = stoi[ch]
            X.append(context)
            y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    y = torch.tensor(y)
    return X,y

import random
random.seed(42)
random.shuffle(words)
n1 = int(len(words)*0.8)
n2 = int(len(words)*0.9)

Xtr,ytr = build_dataset(words[:n1])
Xval,yval = build_dataset(words[n1:n2])
Xte,yte = build_dataset(words[n2:])


In [97]:
Xtr

tensor([[ 0,  0,  0],
        [ 0,  0, 25],
        [ 0, 25, 21],
        ...,
        [15, 12,  4],
        [12,  4,  1],
        [ 4,  1, 14]])

This function, `cmp`, is used to compare the gradients computed by PyTorch’s autograd system with the manually computed gradients. It helps verify the correctness of the custom gradient calculations. Let’s break it down line by line:

```python
def cmp(s, dt, t):
```
- Defines the function `cmp` with three parameters:
  - `s`: A string, typically a label or description for the comparison.
  - `dt`: The manually computed gradient (from your own calculations).
  - `t`: A PyTorch tensor with `requires_grad=True`, whose `.grad` attribute stores the gradient computed by PyTorch.

```python
    ex = torch.all(dt == t.grad).item()
```
- `torch.all(dt == t.grad)`: This checks if all elements in `dt` and `t.grad` are **exactly equal**.
- `.item()`: Converts the result (a tensor) into a Python boolean (`True` or `False`).
- Stores the result in `ex`. If `ex` is `True`, it means the gradients are exactly the same.

```python
    app = torch.allclose(dt, t.grad)
```
- `torch.allclose(dt, t.grad)`: This checks if `dt` and `t.grad` are approximately equal, allowing for small numerical differences due to floating-point precision.
- Stores the result in `app`.

```python
    maxdiff = (dt - t.grad).abs().max().item()
```
- `(dt - t.grad)`: Computes the element-wise difference between the manually computed gradient and PyTorch’s gradient.
- `.abs()`: Takes the absolute value of the differences.
- `.max()`: Finds the maximum absolute difference.
- `.item()`: Converts the result into a Python scalar.
- Stores the result in `maxdiff`. A large value here suggests significant discrepancies between the gradients.

```python
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')
```
- Uses formatted printing to display:
  - `s`: The label, left-aligned in a 15-character wide field.
  - `exact`: Whether the gradients are exactly equal (`True` or `False`).
  - `approximate`: Whether they are approximately equal (`True` or `False`).
  - `maxdiff`: The maximum absolute difference between the gradients.

### Summary:
This function is useful for debugging and validating gradient calculations in custom deep learning implementations, such as when implementing backpropagation manually. If `ex` is `True`, the gradients match exactly; if `app` is `True`, they are close enough for practical purposes; if `maxdiff` is large, there may be an error in the manual computation.

In [98]:
# function to compare pytorch gradients with our own calculated gradiets
def cmp(s,dt,t):
    ex = torch.all(dt==t.grad).item()
    app = torch.allclose(dt,t.grad)
    maxdiff = (dt -t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

    The key question is:

### **"What should we multiply the weights by so that the variance of activations remains 1?"**

This ensures stable forward propagation, preventing activations from **exploding** or **vanishing**.

---

### **Step-by-Step Derivation**
Let’s say we have an input **$x$** with variance $\text{Var}(x)$ and a weight matrix $W$. The output is:

$$
y = Wx
$$

Now, we want to determine the **correct scaling for $W$** so that $\text{Var}(y) = 1$.

#### **Key Concept: Scaling a Normally Distributed Variable**

If $X∼N(0,1)$, then multiplying it by a scalar k results in:

 $Y=kX$

where $Y$ follows a new normal distribution:

$Y∼N(0,k^2)$

So the **variance changes** from **1** to **$k^2$**.

#### **1. Compute Variance of $y$**
For a single neuron, assuming $W$ and $x$ are independent and zero-mean:

$$
\text{Var}(y) = \text{Var}(Wx)
$$

Since $y$ is a sum of weighted inputs:

$$
y = \sum_{i=1}^{n_{\text{in}}} W_i x_i
$$

Using variance properties:

$$
\text{Var}(y) = \sum_{i=1}^{n_{\text{in}}} \text{Var}(W_i x_i)
$$

Since $\text{Var}(W_i x_i) = \text{Var}(W_i) \cdot \text{Var}(x_i)$, and assuming  $x \sim \mathcal{N}(0,1)$ (standardized inputs), we get:

$$
\text{Var}(y) = n_{\text{in}} \cdot \text{Var}(W)
$$

To **keep $\text{Var}(y) = 1$**, we solve:

$$
n_{\text{in}} \cdot \text{Var}(W) = 1
$$

$$
\text{Var}(W) = \frac{1}{n_{\text{in}}}
$$

Thus, **to achieve this variance, \( W \) should be initialized as:**

$$
W \sim \mathcal{N}(0, \frac{1}{n_{\text{in}}})
$$

or equivalently:

$$
W = \text{torch.randn(...)} \times \frac{1}{\sqrt{n_{\text{in}}}}
$$

This is **Xavier initialization**.

---

### **2. Why Multiply by $\frac{5}{3}$?**
For **Tanh**, the optimal scaling factor is slightly different.
- **Tanh squashes large inputs**, leading to **smaller gradients**.
- Empirical studies found that **scaling weights by $5/3$** helps keep activations in a useful range.

So we modify our initialization:

$$
W = \text{torch.randn(...)} \times \frac{5/3}{\sqrt{n_{\text{in}}}}
$$

---

### **Final Answer**
We ask:
✅ *What should we multiply weights by so that* **Var(y) = 1**?

And the answer is:
✅ **Multiply by $\frac{5}{3} / \sqrt{n_{\text{in}}}$ to maintain proper variance for Tanh activations.** 🚀

In [99]:
n_embed = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embed),     generator=g)

# layer 1
w1 = torch.randn((n_embed*block_size,n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5)
b1 = torch.randn((n_hidden,), generator=g) * 0.1

# layer 2
w2 = torch.randn((n_hidden,vocab_size), generator=g) * 0.1
b2 = torch.randn((vocab_size,), generator=g) * 0.1

# BatchNorm params
bngain = torch.randn((1,n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1,n_hidden)) * 0.1

params = [C,w1,w2,b1,b2,bngain,bnbias]
print(sum(p.nelement() for p in params))
for p in params:
    p.requires_grad = True


12297


In [100]:
batch_size = 32
n = batch_size
ix = torch.randint(0,Xtr.shape[0],(batch_size,),generator=g)
xb,yb = Xtr[ix],ytr[ix] # batch x, y

$h_{\text{pre}} = X W + b$

In [101]:
emb = C[xb]
embcat = emb.view(emb.shape[0],-1)
# linear layer 1
hprebn = embcat @ w1 + b1 # hidden layer pre activation

## Batch Mean
$\mu_B = \frac{1}{n} \sum_{i=1}^{n} h_{\text{pre}, i}$

where ( n ) is the batch size.


Subtracts the mean from each activation.

$\hat{h}{\text{pre}, i} = h{\text{pre}, i} - \mu_B$

In [102]:
# BatchNorm layer
bnmeani = 1/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2

### Compute the Variance (Batch Variance)

$\sigma_B^2 = \frac{1}{n-1} \sum_{i=1}^{n} (h_{\text{pre}, i} - \mu_B)^2$

The (n-1) instead of n is because of Bessel’s correction, which gives an unbiased estimate of variance for small batch sizes.

### Compute Inverse Standard Deviation:
( +1e-5 ): Small epsilon to prevent division by zero.

$\text{std}_B = \sqrt{\sigma_B^2 + \epsilon}$

$\text{std}_B^{-1} = \frac{1}{\sqrt{\sigma_B^2 + \epsilon}}$

### Normalize the Activations

$\hat{h}i = \frac{h{\text{pre}, i} - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$

Now, each feature in the batch has mean 0 and variance 1.




In [103]:
bnvar =  1 /(n-1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv

Scale and Shift (Learnable Parameters)
Final transformation:

$h_{\text{post}, i} = \gamma \hat{h}_i + \beta$

This allows the model to learn back any necessary scaling and shifting, so the network doesn't lose expressiveness after normalization.

In [104]:
hpreact = bngain * bnraw + bnbias
# non linearity
h = torch.tanh(hpreact) # hidden layer
# linear layer 2
logits = h @ w2 + b2  # output layer



In [105]:
# crosst entropy loss (same as F.cross_entrop(logits,yb))
logit_maxes = logits.max(1,keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1,keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n),yb].mean()

## **What Are We Trying to Achieve?**
We are computing the **cross-entropy loss** for classification.

### **Cross-Entropy Loss Definition**
For a given sample $i$, if the logits (raw scores before softmax) are $z_i$, and the correct class index is $y_i$, the cross-entropy loss is:

$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \log p_{i, y_i}
$$

where:
- $p_{i, y_i}$ is the probability assigned to the correct class.
- $p_i = \text{softmax}(z_i)$, which converts logits into probabilities.

So, our goal is to compute:
1. **Softmax probabilities**: $p_i = \frac{e^{z_{i,j}}}{\sum_j e^{z_{i,j}}}$
2. **Log probability of the correct class**: $\log p_{i, y_i}$
3. **Take the negative mean** to get the loss.


### **Step 1: Normalize Logits (For Numerical Stability)**
- Logits can be large, and exponentiating them directly can cause numerical overflow.
- To avoid this, we **subtract the max logit** from each row.
- This doesn’t change softmax results but keeps numbers in a stable range.

Mathematically:
$$
\tilde{z}_{i,j} = z_{i,j} - \max_j z_{i,j}
$$

### **Step 2: Compute Exponentials (Softmax Numerator)**
- We exponentiate the **normalized** logits:
$$
e^{\tilde{z}_{i,j}}
$$

---

### **Step 3: Compute Softmax Denominator**

- This computes the **sum of exponentials** for each row (batch sample):
$$
S_i = \sum_j e^{\tilde{z}_{i,j}}
$$

---

### **Step 4: Compute Softmax Probabilities**

- Instead of dividing (which is slower), we multiply by the inverse:
$$
p_{i,j} = \frac{e^{\tilde{z}_{i,j}}}{S_i}
$$

This gives us the **softmax probabilities**.

---

### **Step 5: Compute Log Probabilities**

- Since softmax values are probabilities, we take their log:
$$
\log p_{i,j} = \log \left( \frac{e^{\tilde{z}_{i,j}}}{S_i} \right) = \tilde{z}_{i,j} - \log S_i
$$
- This is the log-probability of each class.

---

### **Step 6: Compute Loss for Correct Classes**
```python
loss = -logprobs[range(n), yb].mean()
```
- `logprobs[range(n), yb]` extracts the **log probability of the correct class** for each sample.
- Taking the **negative mean** gives us:
$$
\mathcal{L} = -\frac{1}{N} \sum_{i=1}^{N} \log p_{i, y_i}
$$
which is exactly the **cross-entropy loss**.

---



In [106]:
# pytorch backward pass
for p in params:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.8275, grad_fn=<NegBackward0>)

In [112]:
-logprobs[range(n),yb].mean()

tensor(3.8275, grad_fn=<NegBackward0>)

In [110]:
yb

tensor([ 1, 12,  0,  5,  9, 18,  9, 16,  1,  0,  0,  1,  0,  0, 14, 12,  0,  0,
         0,  8, 25,  5,  0, 20, 19, 15, 12, 22, 22,  2, 21, 18])

In [ ]:
# loss = -1/na + -1/nb + -1/nc
# dloss/da = -1/n


$$
\text{loss} = - \frac{1}{n} \sum_{i=1}^{n} \text{logprobs}_{i, Yb_i}
$$

$$
\frac{\partial \text{loss}}{\partial \text{logprobs}_{i, Yb_i}} = - \frac{1}{n}
$$



In [113]:
# excercise 1 backprop through the whole thing manually
# backpropagating thrpough exactly all the varaiables
# as they are defined in forward pass one by one

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n),yb] = -1.0/n
cmp('logprobs',dlogprobs,logprobs)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


$$
\frac{\partial \text{loss}}{\partial \text{probs}_{i, j}} = \frac{\partial \text{loss}}{\partial \text{logprobs}_{i, j}} \cdot \frac{\partial \text{logprobs}_{i, j}}{\partial \text{probs}_{i, j}}
$$

The derivative of `logprobs` with respect to `probs` is:

$$
\frac{\partial \text{logprobs}_{i, j}}{\partial \text{probs}_{i, j}} = \frac{1}{\text{probs}_{i, j}}
$$

Thus, the gradient becomes:

$$
\frac{\partial \text{loss}}{\partial \text{probs}_{i, j}} = - \frac{1}{n} \cdot \frac{1}{\text{probs}_{i, Yb_i}}
$$

This simplifies to:

$$
\frac{\partial \text{loss}}{\partial \text{probs}_{i, j}} = - \frac{1}{n \cdot \text{probs}_{i, Yb_i}} \quad \text{for} \quad j = Yb_i
$$

In [120]:
dprobs = (1.0/probs) * dlogprobs
cmp('probs',dprobs,probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


In [121]:
1.0/probs * dlogprobs

tensor([[ 0.0000, -0.1390,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000, -1.6662,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [-1.3661,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.6258,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

$$
\text{loss} = -\frac{1}{n} \sum_{i=1}^n \log(\text{probs}_{i, \text{yb}_i})
$$

Where $\text{probs}_i = \frac{\text{counts}_i}{\text{counts\_sum}_i} = \text{counts}_i \cdot \text{counts\_sum\_inv}_i$.


$$
\frac{\partial \text{loss}}{\partial \text{counts\_sum\_inv}_i}
$$

$$
\frac{\partial \text{loss}}{\partial \text{logprobs}_i} = -\frac{1}{n}
$$

$$
\frac{\partial \text{logprobs}_i}{\partial \text{probs}_i} = \frac{1}{\text{probs}_i}
$$

$$
\frac{\partial \text{probs}_i}{\partial \text{counts\_sum\_inv}_i} = \text{counts}_i
$$


Using the chain rule, we get:

$$
\frac{\partial \text{loss}}{\partial \text{counts\_sum\_inv}_i} = \frac{\partial \text{loss}}{\partial \text{logprobs}_i} \cdot \frac{\partial \text{logprobs}_i}{\partial \text{probs}_i} \cdot \frac{\partial \text{probs}_i}{\partial \text{counts\_sum\_inv}_i}
$$

Substituting the derivatives:

$$
\frac{\partial \text{loss}}{\partial \text{counts\_sum\_inv}_i} = -\frac{1}{n} \cdot \frac{1}{\text{probs}_i} \cdot \text{counts}_i
$$

So, the final expression for the derivative of the loss with respect to $\text{counts\_sum\_inv}_i$ is:

$$
\frac{\partial \text{loss}}{\partial \text{counts\_sum\_inv}_i} = -\frac{1}{n} \cdot \frac{\text{counts}_i}{\text{probs}_i}
$$



In [128]:
dcounts_sum_inv = (counts * dprobs).sum(1,keepdim=True)
cmp('counts_sum_inv',dcounts_sum_inv,counts_sum_inv)


counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


Yes, **exactly**! Here's the detailed reasoning:

### Broadcasting and Gradient Flow

When we perform operations like normalization, the **inverse sum (`counts_sum_inv`)** is computed by summing across the features (axis 1) of each sample. Since this normalization factor is applied to each class in the sample, it gets **broadcasted** across the features (classes). This means that the same inverse sum (`counts_sum_inv`) is applied to all the features (classes) of a given sample.

In terms of backpropagation:

1. **Broadcasting**:
   - The inverse sum (`counts_sum_inv`) is a tensor of shape `[32, 1]` (i.e., one value per sample).
   - When you use this value to normalize across all the features (27 classes), it is **broadcasted** to a tensor of shape `[32, 27]`. So, for each sample, you apply the same normalization factor to all of its classes.

   This ensures that each class's probability is scaled by the same normalization factor for its corresponding sample.

2. **Gradient Flow**:
   - When performing backpropagation, each class's predicted probability (`probs`) contributes to the loss. The gradients of the loss w.r.t. the probabilities (`dprobs`) are calculated for each class.

   - However, since `counts_sum_inv` is shared across all the classes of a sample, you need to account for the fact that changing **one class's count** affects the **entire sum** of counts for that sample.

3. **Summing Gradients**:
   - Since `counts_sum_inv` is applied uniformly across all classes in a sample, the gradient of the loss with respect to `counts_sum_inv` is **the same for all classes** in a given sample. But, each class has a different contribution to this gradient, and thus we need to **sum** the gradients from all the features (classes) to compute how much the `counts_sum_inv` needs to be adjusted.

   - **Why summing?** The reason we sum is that **each class contributes** to the overall sum (and inverse sum). Thus, we need to aggregate the effects of all the classes on the gradient with respect to `counts_sum_inv`.

### Visualizing the Gradient Calculation

Let's say we have:

- `counts_sum_inv[i, :]` is a **shared normalization factor** for the `i`-th sample across all its classes.
- `dprobs[i, :]` is the gradient of the loss with respect to the probabilities of the `i`-th sample.

Now, when we compute the gradient of the loss w.r.t. `counts_sum_inv`, we need to consider that the inverse sum affects **all the features (classes)**. So, we compute:

$$
d\text{counts\_sum\_inv} = (\text{counts} \cdot d\text{probs}).\text{sum}(1, \text{keepdim=True})
$$

For each sample:
- The gradient with respect to the inverse sum is the **sum of the gradients** across all features. Each feature's gradient is multiplied by its corresponding count, and then summed across all features to aggregate the effects of each class on the normalization.

### Conclusion:
Yes, `counts_sum_inv` is broadcasted across the features for each sample, and you sum the gradients from all classes because they are all impacted by the same inverse sum. This ensures that the gradient w.r.t. `counts_sum_inv` correctly aggregates the effects of all features (classes) for each sample.

In [130]:
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum_inv',dcounts_sum_inv,counts_sum_inv)
cmp('counts_sum',dcounts_sum,counts_sum)

counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


In [132]:
# a11 a12 a13 ---> b1( = a11 + a12 + a13)
# a21 a22 a23 ---> b2( = a21 + a22 + a23)
# a21 a32 a33 ---> b3( = a31 + a32 + a33)



In [131]:
dcounts+= torch.ones_like(counts) * dcounts_sum
cmp('counts',dcounts,counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0
